### 專案名稱: 第四屆機器學習百日馬拉松
### 功能描述: 第101~103天作業
### 版權所有: 
### 程式撰寫: Dunk  
### 撰寫日期：2020/07/15
### 改版日期:  
### 改版備註:

#### 套用第三屆的寫法了

In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import Flatten, Dense
from keras.models import Model
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
DATASET_PATH  = './image_data/train'
TEST_PATH = './image_data/test'
OUTPUT_PATH = './working'
NUM_CLASSES = 5
NUM_EPOCHS = 32
SEED = 77
# saved model
WEIGHTS_FINAL = 'model-InceptionResNetV2.h5'
categories=os.listdir(DATASET_PATH)

In [3]:
# Image preprocess
train_datagen = ImageDataGenerator(rotation_range=30, width_shift_range=0.125, height_shift_range=0.125, zoom_range=0.125, horizontal_flip=True,
                                   validation_split=0.2, rescale=1. / 255)
train_batches = train_datagen.flow_from_directory(DATASET_PATH, subset = 'training', seed = SEED)
valid_batches = train_datagen.flow_from_directory(DATASET_PATH, subset = 'validation', seed = SEED)
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))

Found 2260 images belonging to 5 classes.
Found 563 images belonging to 5 classes.
Class #0 = daisy
Class #1 = dandelion
Class #2 = rose
Class #3 = sunflower
Class #4 = tulip


In [4]:
# Build Model
net = InceptionResNetV2(include_top=False, input_shape=(256, 256, 3))
x = net.output
x = Flatten()(x)
# Add Dense layer, each probability by softmax
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)
net_final = Model(inputs=net.input, outputs=output_layer)
net_final.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
# Train
net_final.fit_generator(train_batches, validation_data = valid_batches, epochs = NUM_EPOCHS)
# Store Model
net_final.save(WEIGHTS_FINAL)

Epoch 1/32
71/71 [==============================] - 3053s 43s/step - loss: 1.6593 - accuracy: 0.6898 - val_loss: 17696.2031 - val_accuracy: 0.1812
Epoch 2/32
71/71 [==============================] - 3208s 45s/step - loss: 1.3479 - accuracy: 0.7137 - val_loss: 191481.5000 - val_accuracy: 0.4671
Epoch 3/32
71/71 [==============================] - 3070s 43s/step - loss: 1.0378 - accuracy: 0.7673 - val_loss: 1.0209 - val_accuracy: 0.6430
Epoch 4/32
71/71 [==============================] - 3031s 43s/step - loss: 0.6994 - accuracy: 0.8164 - val_loss: 481.0917 - val_accuracy: 0.2540
Epoch 5/32
71/71 [==============================] - 3021s 43s/step - loss: 0.6419 - accuracy: 0.7951 - val_loss: 0.2421 - val_accuracy: 0.7833
Epoch 6/32
71/71 [==============================] - 3002s 42s/step - loss: 2.5765 - accuracy: 0.7363 - val_loss: 405615.9688 - val_accuracy: 0.1705
Epoch 7/32
71/71 [==============================] - 3049s 43s/step - loss: 2.0387 - accuracy: 0.5956 - val_loss: 27893.2168 - 

In [6]:
from keras.preprocessing import image
out = np.array(['id', 'flower_class'])
testfiles=os.listdir(TEST_PATH)
for testfile in testfiles:
    filename = testfile.split('.')[0]
    img = image.load_img(TEST_PATH+'/'+testfile,target_size=(256, 256))
    if img is None:
        continue
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x = x /255.
    pred = net_final.predict(x)[0]
    tof=np.argmax(pred)
    out = np.vstack((out,[filename, tof]))

pd.DataFrame(out).to_csv(OUTPUT_PATH+'/prediction.csv',index=False,header=False)